<a href="https://colab.research.google.com/github/Pranjal-droi/urban-parking-pricing/blob/main/dynamicpricingmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopy bokeh --quiet

In [ ]:
!pip install pathway


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
import pathway as pw
from geopy.distance import geodesic
output_notebook()

In [ ]:
!wget -O dataset.csv "https://raw.githubusercontent.com/Pranjal-droi/urban-parking-pricing/main/dataset.csv"

import pandas as pd
df = pd.read_csv('/content/dataset.csv')
print("✅ Dataset Loaded:", df.shape)
df.head()



In [ ]:
@pw.udf
def compute_demand_based_price(occupancy, capacity, queue, traffic, is_special, vehicle_type):
    # Default weights
    alpha, beta, gamma, delta, epsilon = 2.0, 1.0, 1.5, 2.0, 1.0
    base_price = 10.0
    lam = 0.75

    # Encoding
    traffic_map = {'low': 1, 'medium': 2, 'high': 3}
    vehicle_map = {'car': 1.0, 'bike': 0.7, 'truck': 1.5}

    try:
        traffic_level = traffic_map.get(traffic, 2)
        vehicle_weight = vehicle_map.get(vehicle_type, 1.0)
        occ_ratio = occupancy / capacity if capacity > 0 else 0

        # Raw demand calculation
        demand = (
            alpha * occ_ratio +
            beta * queue -
            gamma * traffic_level +
            delta * is_special +
            epsilon * vehicle_weight
        )

        # Normalize demand using tanh as a smooth function (avoid MinMax fit)
        norm_demand = np.tanh(demand / 10)
        price = base_price * (1 + lam * norm_demand)
        return round(min(max(price, 5), 20), 2)  # Clamp between $5–$20
    except:
        return base_price

# ================================
# 🧾 4. DEFINE DATA SCHEMA
# ================================
class ParkingInputSchema(pw.Schema):
    SystemCodeNumber: str
    Capacity: int
    Occupancy: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

# ================================
# 🔄 5. READ STREAM & APPLY LOGIC
# ================================
input_table = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingInputSchema,
    mode="static",  # stream simulation from static CSV
    autocommit_duration_ms=1000
)

# Add new column with computed prices
result = input_table.select(
    SystemCodeNumber=input_table.SystemCodeNumber,
    Occupancy=input_table.Occupancy,
    Capacity=input_table.Capacity,
    Price=compute_demand_based_price(
        input_table.Occupancy,
        input_table.Capacity,
        input_table.QueueLength,
        input_table.TrafficConditionNearby,
        input_table.IsSpecialDay,
        input_table.VehicleType
    )
)

# ================================
# 💾 6. WRITE OUTPUT
# ================================
pw.io.csv.write(result, "prices_output.json")

# ================================
# ▶️ 7. RUN
# ================================
pw.run()

In [ ]:
df['BasePrice'] = 10.0
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']

In [ ]:
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)


In [ ]:
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)


In [ ]:
vehicle_map = {'car': 1.0, 'bike': 0.7, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_map)

In [ ]:
df['TrafficLevel'].fillna(2, inplace=True)
df['VehicleTypeWeight'].fillna(1.0, inplace=True)

In [ ]:
alpha1 = 5.0
df['LinearPrice'] = np.nan

for lot, group in df.groupby('SystemCodeNumber'):
    group = group.sort_values(by=['ID']) # Sorting by ID as a proxy for time
    prices = [10.0]
    for i in range(1, len(group)):
        occ_ratio = group.iloc[i]['OccupancyRatio']
        next_price = prices[-1] + alpha1 * occ_ratio
        prices.append(next_price)
    df.loc[group.index, 'LinearPrice'] = prices

In [ ]:
alpha_d, beta_d, gamma_d, delta_d, epsilon_d = 2.0, 1.0, 1.5, 2.0, 1.0
df['RawDemand'] = (
    alpha_d * df['OccupancyRatio'] +
    beta_d * df['QueueLength'] -
    gamma_d * df['TrafficLevel'] +
    delta_d * df['IsSpecialDay'] +
    epsilon_d * df['VehicleTypeWeight']
)

scaler = MinMaxScaler()
df['NormalizedDemand'] = scaler.fit_transform(df[['RawDemand']])

lambda_d = 0.75
df['DemandPrice'] = df['BasePrice'] * (1 + lambda_d * df['NormalizedDemand'])
df['DemandPrice'] = df['DemandPrice'].clip(5, 20)

In [ ]:
lot_locations = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first()

In [ ]:
distance_matrix = pd.DataFrame(index=lot_locations.index, columns=lot_locations.index)
for lot1 in lot_locations.index:
    for lot2 in lot_locations.index:
        coord1 = (lot_locations.loc[lot1, 'Latitude'], lot_locations.loc[lot1, 'Longitude'])
        coord2 = (lot_locations.loc[lot2, 'Latitude'], lot_locations.loc[lot2, 'Longitude'])
        distance_matrix.loc[lot1, lot2] = geodesic(coord1, coord2).km

In [ ]:
price_lookup = {}
for (lot, id), group in df.groupby(['SystemCodeNumber', 'ID']):
    price_lookup[(lot, id)] = dict(zip(group['SystemCodeNumber'], group['DemandPrice']))

In [ ]:
def adjust_price(row):
    lot = row['SystemCodeNumber']
    time_key = (row['LastUpdatedDate'], row['LastUpdatedTime'])
    own_price = row['DemandPrice']

    # Nearby within 1km
    nearby_lots = distance_matrix.loc[lot][distance_matrix.loc[lot].astype(float) < 1.0].index.tolist()
    nearby_lots = [l for l in nearby_lots if l != lot]

    competitor_prices = [price_lookup.get(time_key, {}).get(l) for l in nearby_lots]
    competitor_prices = [p for p in competitor_prices if p is not None]

    if not competitor_prices:
        return own_price

    avg_comp = np.mean(competitor_prices)

    if row['Occupancy'] >= row['Capacity'] and avg_comp < own_price:
        return max(own_price - 1.0, 5)
    elif avg_comp > own_price:
        return min(own_price + 1.0, 20)
    return own_price

In [ ]:
lot_locations = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first()
distance_matrix = pd.DataFrame(index=lot_locations.index, columns=lot_locations.index)

for lot1 in lot_locations.index:
    for lot2 in lot_locations.index:
        coord1 = (lot_locations.loc[lot1, 'Latitude'], lot_locations.loc[lot1, 'Longitude'])
        coord2 = (lot_locations.loc[lot2, 'Latitude'], lot_locations.loc[lot2, 'Longitude'])
        distance_matrix.loc[lot1, lot2] = geodesic(coord1, coord2).kilometers

In [ ]:
price_lookup = {}
for (lot, id), group in df.groupby(['SystemCodeNumber', 'ID']):
    price_lookup[(lot, id)] = dict(zip(group['SystemCodeNumber'], group['DemandPrice']))

In [ ]:
def adjust_price(row):
    lot = row['SystemCodeNumber']
    # Use 'ID' as a proxy for time since LastUpdatedDate and LastUpdatedTime are not available
    time_key = (lot, row['ID'])
    own_price = row['DemandPrice']

    # Nearby within 1km
    # Ensure distance_matrix values are float for comparison
    nearby_lots = distance_matrix.loc[lot][distance_matrix.loc[lot].astype(float) < 1.0].index.tolist()
    nearby_lots = [l for l in nearby_lots if l != lot]

    competitor_prices = [price_lookup.get(time_key, {}).get(l) for l in nearby_lots]
    competitor_prices = [p for p in competitor_prices if p is not None]

    if not competitor_prices:
        return own_price

    avg_comp = np.mean(competitor_prices)

    # Adjust price based on occupancy and competitor prices
    if row['Occupancy'] >= row['Capacity'] and avg_comp < own_price:
        return max(own_price - 1.0, 5)  # Lower price if full and competitors are cheaper
    elif avg_comp > own_price:
        return min(own_price + 1.0, 20)  # Increase price if competitors are more expensive
    return own_price

df['CompetitivePrice'] = df.apply(adjust_price, axis=1)

In [ ]:
from bokeh.palettes import Category10
def plot_prices(lot_id):
    lot_df = df[df['SystemCodeNumber'] == lot_id].sort_values(by=['ID']) # Sort by ID as a proxy for time

    x = list(range(len(lot_df))) # Convert range to a list
    p = figure(title=f"Real-Time Prices for {lot_id}", x_axis_label='Time (by ID)', y_axis_label='Price ($)', width=800)
    p.line(x, lot_df['LinearPrice'], legend_label="Linear", color=Category10[3][0], line_width=2)
    p.line(x, lot_df['DemandPrice'], legend_label="Demand", color=Category10[3][1], line_width=2)
    p.line(x, lot_df['CompetitivePrice'], legend_label="Competitive", color=Category10[3][2], line_width=2)
    p.legend.location = "top_left"
    return p

# Show sample lot plot
lot_code = df['SystemCodeNumber'].unique()[0]
show(plot_prices(lot_code))

In [ ]:
import pandas as pd

pathway_results_df = pd.read_csv("pathway_output.csv")
print("✅ Pathway results loaded:")
display(pathway_results_df.head())